In [1]:
import cv2
import time
import math
import torch
import random
import torchreid
import numpy as np
from glob import glob
from PIL import Image
from threading import Thread
from datetime import datetime
from math import hypot as Dist
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## Person_ReIdentification 

In [3]:
# datamanager = torchreid.data.ImageDataManager(
#     root="reid-data",
#     sources="market1501",
#     targets="market1501",
#     height=256,
#     width=128,
#     batch_size_train=32,
#     batch_size_test=100,
#     transforms=["random_flip", "random_crop"]
# )

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize((256, 128))
#     transforms.Resize((160, 64))
])

In [4]:
myModel = torchreid.models.build_model(
#     name="hacnn",
    name="xception",
    num_classes=751, #datamanager.num_train_pids
    loss="softmax",
    pretrained=True
)

In [5]:
# checkpoint = torch.load('./model.pth.tar-100')
checkpoint = torch.load('./model.pth.tar-80')
# checkpoint = torch.load('./log5/HaCNN/model/model.pth.tar-120')
myModel.load_state_dict(checkpoint['state_dict'])

myModel = myModel.to(device)
myModel.eval()

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv2d(128, 128, kernel_size=(3

## Person Detection Model 

In [6]:
things_model = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5x6.pt',device=f'{device}:0' if device=='cuda' else f'{device}')
%matplotlib inline

Using cache found in C:\Users\a.nasimi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-2 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


# Camera Setting

In [7]:
class ThreadedCamera(object):
    def __init__(self, source='cam:Ashkan123@192.168.172.199:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

# Handeling Functions

## PreValues

In [8]:
counter = 0
count_skipped = 0

thresh = 8.5
thresh_param = 3
thresh_on_estimate = int(5e2)

old_frame,last_labels = {},[]
counter,skipped_frame,skipped_count = 0,0,0

ROI_Region,ROI_Person = [0,0,0,0],[0,0,0,0]
img_original,database,database_unknowns,rgb_colors = None,[],[],{}

## Functions 

### Person Detection Functions 

In [9]:
def Pre_Proc_Light(img):
    lightness = np.mean(img)
    if lightness < 95:
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    else: return img

In [10]:
# from pathlib import Path
# import tensorflow as tf
# from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

# bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))

# def Blurring(image):

# #     image_array = tf.keras.preprocessing.image.img_to_array(image)
#     result = bodypix_model.predict_single(image)

#     mask = result.get_mask(threshold=0.20)
#     res = cv2.bitwise_and(image,image,mask = np.uint8(mask))
    
#     blur = cv2.blur(image,(45,45),0)
#     out = image.copy()

#     sharpen_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
#     sharpen = cv2.filter2D(out, -1, sharpen_kernel)

#     sharpen[res==0] = blur[res==0]
 
#     return sharpen
    

# #     # colored mask (separate colour for each body part)
# #     colored_mask = result.get_colored_part_mask(mask)

In [11]:
def Person_Detection(img,img_orig,ROI_Region):
    imgs = []
    save_flag = True
    img_original = img_orig.copy()
    
    now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
    img = np.array(Image.fromarray(img))
    img = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)
    
    with torch.no_grad(): results = things_model(img)

    if len(results.xyxy[0]) == 0: pass
    else:
        img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_BGR2RGB)
        
        for indx,res in enumerate(results.xyxy[0]):
            
            predict = results.pandas().xyxy[0]['name'][indx]
            
            if predict == 'person': pass
            else: continue

            res = np.array(res.detach().cpu())
            if res[4]*100 < 45: save_flag = False
            else: 
                x = round(res[0])
                y = round(res[1])
                w = round(res[2])
                h = round(res[3])
                
                imgs.append((img[y:h,x:w],[x,y,h,w]))
        
    return imgs

In [12]:
def Crop_Persons(imgs,new_frame):
    indx_ = 0
    for img,cordinations in imgs:
        x = cordinations[0]
        y = cordinations[1]
        h = cordinations[2]
        w = cordinations[3]

#         img_ = Blurring(img)
        img_ = img.copy()

        if img_.shape[0] < 300: continue
        elif img_.shape[1] < 140: continue
        else:          
            new_frame[indx_] = {
                'Img':img_,
                'Status':'-',
                'Label':'Unknown',
                'Coordination':[x,y,w,h],
                'Last_PID_Run':datetime.now()
            }
            indx_ += 1
            
    return new_frame

### Person Re-Identification 

In [13]:
def Check_Similarity(distns,thresh):
    try: indx,dist = min(distns, key=lambda x: x[1])
    except ValueError: dist = 9e9

    if dist < thresh: return indx,dist,True
    else: return 'Unknown',None,False

In [14]:
def Person_ReIdentification(img,flag_known=True,flag_unknown=False):
    img = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2RGB)
    img = Pre_Proc_Light(img)
    img_ = transform(img)
    x = myModel(img_.unsqueeze(0).float().cuda()).detach().cpu()
    
    if flag_known:
        distns = []
        for data in database:
            for embd in data['Embds']:
                dist = torch.cdist(x, embd, p=2.0)[0][0]
                distns.append((data['Id'],dist))
        return distns,x
    else: pass
            
    if flag_unknown:
        distns = []
        for data in database_unknowns:
            for embd in data['Embds']:
                dist = torch.cdist(x, embd, p=2.0)[0][0]
                distns.append((data['Id'],dist))
        return distns,x
    else: pass
    
    return x

In [15]:
def Identify_Persons(persons,old_frame_info,labels):
    for person_num in persons:
        distns,emb = Person_ReIdentification(persons[person_num]['Img'],flag_known=True,flag_unknown=False)
        label,dist,flag = Check_Similarity(distns,thresh)

        if flag: 
            persons[person_num],_,_,_,_,labels,old_frame,persons,flag_state = Check_And_Go(persons[person_num],label,emb,'Known',dist,labels,old_frame_info,persons)
            if flag_state:
                labels.append(label)
                persons[person_num]['Label'] = label
                persons[person_num]['Status'] = 'Identified'
                persons[person_num]['Last_PID_Run'] = datetime.now()
            else: continue

        else:
            distns,emb = Person_ReIdentification(persons[person_num]['Img'],flag_known=False,flag_unknown=True)
            label,dist,flag = Check_Similarity(distns,thresh)

            if flag: 
                persons[person_num],_,_,_,_,labels,old_frame,persons,flag_state = Check_And_Go(persons[person_num],label,emb,'Unknown',dist,labels,old_frame_info,persons)
                
                if flag_state:
                    labels.append(label)
                    persons[person_num]['Label'] = label
                    persons[person_num]['Status'] = 'Identified'
                    persons[person_num]['Last_PID_Run'] = datetime.now()
                else: continue

            else:
                label,dist,flag = Check_Similarity(distns,thresh+thresh_param)

                if flag:
                    persons[person_num],_,_,_,_,labels,old_frame,persons,flag_state = Check_And_Go(persons[person_num],label,emb,'Unknown',dist,labels,old_frame_info,persons)
#                     labels,persons,flag_state = Check_And_Go(persons[person_num],label,emb,'Unknown',dist,labels,old_frame_info,persons)
                    if flag_state:
                        labels.append(label)
                        persons[person_num]['Label'] = label
                        persons[person_num]['Status'] = 'Identified'
                        persons[person_num]['Last_PID_Run'] = datetime.now()
                    else: continue

                else: 
                    label = Add_Person_To_DB(emb,False)
                    
                    labels.append(label)
                    persons[person_num]['Label'] = label
                    persons[person_num]['Status'] = 'Identified'
                    persons[person_num]['Last_PID_Run'] = datetime.now()

    return persons

In [16]:
def Check_And_Go(current_person,current_label,current_emb,state,current_dist,prev_labels,old_frame,new_persons):
    if current_label in prev_labels:
        print("1")
        flag_ = False
        for indx in old_frame:
            if current_label == old_frame[indx]['Label']:
                flag_ = True
                break
            else: continue
                 
        if flag_:  # آیا برچسب مذکور در فریم قبلی دیده شده است؟
            print("4")
            if 'Unknown' in current_label:
                label = Add_Person_To_DB(current_emb,False)

                prev_labels.append(label)
                current_person['Label'] = label
                current_person['Status'] = 'Identified'
                current_person['Last_PID_Run'] = datetime.now()

                return current_person,label,current_emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False
            
            else:
                distns,emb = Person_ReIdentification(current_person['Img'],flag_known=False,flag_unknown=True)
                label,dist,flag = Check_Similarity(distns,thresh)

                print(f"1 - current label: {current_label} - label: {label}")

                if flag: return Check_And_Go(current_person,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
                else:
                    label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
                    print(f"1 - current label: {current_label} - label: {label}")

                    if flag: return Check_And_Go(current_person,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
                    else: 
                        print("5")
                        label = Add_Person_To_DB(emb,False)

                        prev_labels.append(label)
                        current_person['Label'] = label
                        current_person['Status'] = 'Identified'
                        current_person['Last_PID_Run'] = datetime.now()

                        return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False         
                        
        else:   
            for indx in new_persons:
                if new_persons[indx]['Label'] == current_label: break
                else: continue
                
            print("10")
            if ('Unknown' in current_label) and not ('Unknown' == current_label):
                print("6")
                distns,emb = Person_ReIdentification(new_persons[indx]['Img'],flag_known=True,flag_unknown=False)
                label,dist,flag = Check_Similarity(distns,thresh)
                
                if flag: print("7")
                else:
                    distns,emb = Person_ReIdentification(new_persons[indx]['Img'],flag_known=False,flag_unknown=True)
                    label,dist,flag = Check_Similarity(distns,thresh)
                    print(f"2 - current label: {current_label} - label: {label}")
                    
                    if flag: pass
                    else: 
                        label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
                        print(f"2 - current label: {current_label} - label: {label}")
                        
                if dist < current_dist:     
                    print("8")
                    label = Add_Person_To_DB(current_emb,False)

                    prev_labels.append(label)
                    current_person['Label'] = label
                    current_person['Status'] = 'Identified'
                    current_person['Last_PID_Run'] = datetime.now()

                    return current_person,label,current_emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False  

                else:
                    print("9")
                    current_person['Label'] = label
                    current_person['Status'] = 'Identified'
                    current_person['Last_PID_Run'] = datetime.now()

                    label = Add_Person_To_DB(emb,False)

                    prev_labels.append(label)
                    new_persons[indx]['Label'] = label
                    new_persons[indx]['Status'] = 'Identified'
                    new_persons[indx]['Last_PID_Run'] = datetime.now()

                    return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False  
                
            else:
                print("10")
                persons_dists = []
                persons_dists.append(current_dist)
                
                distns,_ = Person_ReIdentification(new_persons[indx]['Img'],flag_known=True,flag_unknown=False)
                try: indx,dist = min(distns, key=lambda x: x[1])
                except ValueError: dist = 9e9
                    
                persons_dists.append(dist)   
                    
                arg = np.argmin(persons_dists)
                if arg == 0:
                    current_person['Label'] = label
                    current_person['Status'] = 'Identified'
                    current_person['Last_PID_Run'] = datetime.now()
                    
                    person_to_investigate = new_persons[indx]
                    
                else: person_to_investigate = current_person
                    
                distns,emb = Person_ReIdentification(person_to_investigate['Img'],flag_known=False,flag_unknown=True)
                label,dist,flag = Check_Similarity(distns,thresh)
                print(f"3 - current label: {current_label} - label: {label}")
                if flag: return Check_And_Go(person_to_investigate,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
                else:
                    label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
                    print(f"3 - current label: {current_label} - label: {label}")
                    if flag: return Check_And_Go(person_to_investigate,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
                    else: 
                        label = Add_Person_To_DB(emb,False)
                        
                        prev_labels.append(label)
                        person_to_investigate['Label'] = label
                        person_to_investigate['Status'] = 'Identified'
                        person_to_investigate['Last_PID_Run'] = datetime.now()
                        
                        return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False
    else: 
        print("2")
        prev_labels.append(current_label)
        return current_person,current_label,current_emb,state,current_dist,prev_labels,old_frame,new_persons,True

### Normal Functions 

In [17]:
def Write_On_Image(label,cor,img_original):
    ROI_Region = [0,0,0,0]
    try: color = rgb_colors[label]
    except KeyError:
        r = random.randrange(1, 256, 20)
        g = random.randrange(1, 256, 20)
        b = random.randrange(1, 256, 20)
        color = (r,g,b)
        rgb_colors[label] = color

    img_original = cv2.rectangle(np.array(img_original), (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1],abs(cor[2]-(cor[0])),abs(cor[3]-(cor[1]))), color, 2)
    img_original = cv2.putText(np.array(img_original), f"No: {label}", (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2,cv2.LINE_AA)
    
    return img_original

In [18]:
def Add_Person_To_DB(person_emb,flag,indx=None):
    if flag:
        if indx is None: indx = f'{len(database) + 1}'
        else: pass
        
        data = {
            'Id':f'{indx}',
            'Embds':[person_emb],
            'Last_Update':time.gmtime()
        }
        
        database.append(data)
        return indx
    
    else:
        if indx is None: indx = f'{len(database_unknowns) + 1}'
        else: 
            if 'Unknown ' in indx: indx = indx.replace('Unknown ','')
            else: pass
        
        data = {
            'Id':f'Unknown {indx}',
            'Embds':[person_emb],
            'Last_Update':time.gmtime()
        }
        
        database_unknowns.append(data)
        return f'Unknown {indx}'

In [19]:
def Databse_Update(persons):
    def Time_Passed(current_time,second=2):
        oldsecond = current_time[5]
        currentsecond = time.gmtime()[5]

        if ((currentsecond - oldsecond) >= second): return True
        else: return False
    
    for data in database:
        if Time_Passed(data['Last_Update']):
            flag_ = False
            for indx in persons:
                if persons[indx]['Label'] == data['Id']: 
                    flag_ = True
                    break
                else: continue

            if flag_:
                if len(data['Embds']) > (15+1): data['Embds'].pop(0)
                else: pass

                new_embd = Person_ReIdentification(persons[indx]['Img'],flag_known=False,flag_unknown=False)
                data['Embds'].append(new_embd)
                data['Last_Update'] = time.gmtime()
                
            else: continue
        else: continue

    for data in database_unknowns:
        if Time_Passed(data['Last_Update']):
            flag_ = False
            for indx in persons:
                if persons[indx]['Label'] == data['Id']: 
                    flag_ = True
                    break
                else: continue

            if flag_:
                if len(data['Embds']) > (15+1): data['Embds'].pop(0)
                else: pass
                
                new_embd = Person_ReIdentification(persons[indx]['Img'],flag_known=False,flag_unknown=False)
                data['Embds'].append(new_embd)
                data['Last_Update'] = time.gmtime()
                
            else: continue
        else: continue

In [20]:
def Merge_Frame_Dictionaries(*keywords):
    new_farme,counter = {},0
    
    for key in keywords:
        for indx in key:
            if key[indx]['Status'] == '-': continue
            else:
                new_farme[counter] = key[indx].copy()
                counter += 1  
                
    return new_farme

In [21]:
def Calculate_Speed(*keywords):
    def Dotproduct(v1, v2):
        return sum((a*b) for a, b in zip(v1, v2))

    def Length(v):
        return math.sqrt(Dotproduct(v, v))
    
    def Angle(v1, v2):
        return math.acos(Dotproduct(v1, v2) / (Length(v1) * Length(v2)))

    point1 = [(keywords[0][0] + keywords[0][2]) / 2,(keywords[0][1] + keywords[0][3]) / 2]
    point2 = [(keywords[1][0] + keywords[1][2]) / 2,(keywords[1][1] + keywords[1][3]) / 2]
    
    angle1 = Angle(point1,[1e-100,1e-100])
    angle2 = Angle(point2,[1e-100,1e-100])
    
#     return (angle2 - angle1) / (keywords[2] / 25)
    
    try: return (angle2 - angle1) * (180/np.pi)
    except IndexError: return (angle2 - angle1) * (np.pi / 180)

# Main Design

## Add Person to Known DB 

In [22]:
# counter_ = 0
# for indx,pth in enumerate(glob('./Sample1*')):
#     if counter_ > 16: break
#     img = cv2.imread(pth)
#     imgs = Person_Detection(img,img,ROI_Region)
    
# #     img = Blurring(imgs[0][0])
#     img = imgs[0][0]
#     emb = Person_ReIdentification(img,False,False)
    
#     if indx == 0: indx_ = Add_Person_To_DB(emb,flag=True,indx='Fadaeian')
#     else: Add_Person_To_DB(emb,flag=True,indx=f'{indx_}')
        
#     counter_ += 1

## Evaluation 

In [23]:
streamer = ThreadedCamera('rtsp:Ashkan321@172.16.60.121')

# cap = cv2.VideoCapture('./video_1.mp4')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer = cv2.VideoWriter('./video_1400_test_known_No_Skip_4_speed.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

In [24]:
while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,all_labels = {},[]
    
#     ret,frame = cap.read()    
    frame = streamer.grab_frame()

    if counter != 0 and (counter % 5) == 0: 
        count_skipped += 1
        if count_skipped > 1: 
            counter += 1
            count_skipped = 0
            
        else: continue
            
    else: counter += 1

    try: img_original = frame.copy()
    except AttributeError: break

    imgs = Person_Detection(frame,img_original,ROI_Region)

    if len(imgs) == 0: pass
    else: new_frame = Crop_Persons(imgs,new_frame)
        
    if len(new_frame) == 0:
        
#         writer.write(img_original)
#         cv2.imwrite(f"./Pics/{counter}.jpg",img_original)
        cv2.imshow('Person_ReIdentification', np.array(img_original))
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            cv2.destroyAllWindows()
            streamer.end_frame()
            break  
        else: continue        
        
        print(f"{counter}--**************************")
        continue

    else: pass
    
    if len(old_frame) == 0 or (counter % thresh_on_estimate) == 0:
        new_frame = Identify_Persons(new_frame,[],all_labels)

    elif len(old_frame) <= len(new_frame):
        for i in old_frame:
            distances_compares,flag_ = [],False
            cor_old = [(old_frame[i]['Coordination'][0]+old_frame[i]['Coordination'][2])/2,(old_frame[i]['Coordination'][1]+old_frame[i]['Coordination'][3])/2]
            for j in new_frame:
                cor_current = [(new_frame[j]['Coordination'][0]+new_frame[j]['Coordination'][2])/2,(new_frame[j]['Coordination'][1]+new_frame[j]['Coordination'][3])/2]     

                dist_ = Dist(cor_old[0]-cor_current[0],cor_old[1]-cor_current[1])
                
#                 speed_ = Calculate_Speed([old_frame[i]['Coordination'][0],
#                           old_frame[i]['Coordination'][1],
#                           old_frame[i]['Coordination'][2],
#                           old_frame[i]['Coordination'][3]],[new_frame[j]['Coordination'][0],
#                                                             new_frame[j]['Coordination'][1],
#                                                             new_frame[j]['Coordination'][2],
#                                                             new_frame[j]['Coordination'][3]],1)
                
                
                distances_compares.append(dist_)

            j = np.argmin(distances_compares)
            
            new_frame[j]['Status'] = 'Estimate'
            new_frame[j]['Label'] = old_frame[i]['Label']
            new_frame[j]['Last_PID_Run'] = datetime.now()
            
            all_labels.append(old_frame[i]['Label'])
            
        new_frame_2 = {}
        for indx in new_frame:
            if new_frame[indx]['Status'] == '-': new_frame_2[indx] = new_frame[indx].copy()
            else: continue
        
        new_frame_2 = Identify_Persons(new_frame_2,old_frame,all_labels)
        
        new_frame = Merge_Frame_Dictionaries(new_frame,new_frame_2)
        
    else: 
        for i in new_frame:
            distances_compares,flag_ = [],False
            cor_old = [(new_frame[i]['Coordination'][0]+new_frame[i]['Coordination'][2])/2,(new_frame[i]['Coordination'][1]+new_frame[i]['Coordination'][3])/2]
            for j in old_frame:
                cor_current = [(old_frame[j]['Coordination'][0]+old_frame[j]['Coordination'][2])/2,(old_frame[j]['Coordination'][1]+old_frame[j]['Coordination'][3])/2]     
                
                dist_ = Dist(cor_old[0]-cor_current[0],cor_old[1]-cor_current[1])
#                 speed_ = Calculate_Speed([new_frame[i]['Coordination'][0],
#                                           new_frame[i]['Coordination'][1],
#                                           new_frame[i]['Coordination'][2],
#                                           new_frame[i]['Coordination'][3]],[old_frame[j]['Coordination'][0],
#                                                                             old_frame[j]['Coordination'][1],
#                                                                             old_frame[j]['Coordination'][2],
#                                                                             old_frame[j]['Coordination'][3]],1)
                distances_compares.append(dist_)

            j = np.argmin(distances_compares)

            new_frame[i]['Status'] = 'Estimate'
            new_frame[i]['Label'] = old_frame[j]['Label']
            new_frame[i]['Last_PID_Run'] = datetime.now()
            
            print("Hryyy")
        
    
    for i in new_frame:
        img_original = Write_On_Image(f"{new_frame[i]['Label']}" ,new_frame[i]['Coordination'],img_original)
    
    
    Databse_Update(new_frame)
    old_frame = new_frame.copy()

    
#     writer.write(img_original)
#     cv2.imwrite(f"./Pics/{counter}.jpg",img_original)
    
    cv2.imshow('Person_ReIdentification', np.array(img_original))
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break  
        
    print(f"{counter}**************************")
    
# cap.release()
# writer.release()

130**************************
131**************************
132**************************
133**************************
134**************************
135**************************
136**************************
137**************************
138**************************
139**************************
140**************************
141**************************
142**************************
143**************************
144**************************
145**************************
146**************************
147**************************
148**************************
149**************************
150**************************
151**************************
152**************************
153**************************
154**************************
155**************************
156**************************
157**************************
158**************************
159**************************
160**************************
161**************************
162**************************
163*******

404**************************
405**************************
406**************************
407**************************
408**************************
409**************************
410**************************
411**************************
412**************************
413**************************
414**************************
415**************************
Hryyy
416**************************
2
417**************************
418**************************
419**************************
420**************************
421**************************
422**************************
423**************************
424**************************
425**************************
426**************************
427**************************
428**************************
429**************************
430**************************
431**************************
432**************************
433**************************
Hryyy
434**************************
435**************************
436***********************

676**************************
677**************************
678**************************
679**************************
680**************************
681**************************
682**************************
683**************************
684**************************
685**************************
686**************************
687**************************
688**************************
689**************************
690**************************
691**************************
692**************************
693**************************
694**************************
695**************************
696**************************
697**************************
698**************************
699**************************
700**************************
701**************************
702**************************
703**************************
704**************************
705**************************
706**************************
707**************************
708**************************
709*******

952**************************
953**************************
957**************************
958**************************
959**************************
960**************************
961**************************
962**************************
963**************************
964**************************
965**************************
966**************************
967**************************
968**************************
969**************************
970**************************
971**************************
972**************************
973**************************
974**************************
975**************************
976**************************
977**************************
978**************************
979**************************
980**************************
981**************************
982**************************
983**************************
984**************************
985**************************
986**************************
987**************************
988*******

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\a.nasimi\Anaconda3\envs\AI\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\a.nasimi\Anaconda3\envs\AI\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\a.nasimi\AppData\Local\Temp\ipykernel_7036\2769113122.py", line 15, in update
cv2.error: Unknown C++ exception from OpenCV code


1044**************************


In [ ]:
database

In [ ]:
database_unknowns[0]

In [ ]:
len(database_unknowns[0]['Embds'])

In [ ]:
# def Check_And_Go(current_person,current_label,current_emb,state,current_dist,prev_labels,old_frame,new_persons):
#     if current_label in prev_labels:
#         print("1")
#         flag_ = False
#         for old_indx in old_frame:
#             if current_label == old_frame[old_indx]['Label']:
#                 flag_ = True
#                 break
#             else: continue
                
#         for indx in new_persons:
#             if new_persons[indx]['Label'] == current_label: break
#             else: continue
                    
#         if flag_:  # آیا برچسب مذکور در فریم قبلی دیده شده است؟
#             print("3")
#             cor_current = [(current_person['Coordination'][0]+current_person['Coordination'][2])/2,(current_person['Coordination'][1]+current_person['Coordination'][3])/2]     
#             cor_prev = [(new_persons[indx]['Coordination'][0]+new_persons[indx]['Coordination'][2])/2,(new_persons[indx]['Coordination'][1]+new_persons[indx]['Coordination'][3])/2]
#             cor_ref = [(old_frame[old_indx]['Coordination'][0]+old_frame[old_indx]['Coordination'][2])/2,(old_frame[old_indx]['Coordination'][1]+old_frame[old_indx]['Coordination'][3])/2]
            
#             dist_1 = Dist(cor_ref[0]-cor_prev[0],cor_ref[1]-cor_prev[1])
#             dist_2 = Dist(cor_ref[0]-cor_current[0],cor_ref[1]-cor_current[1])
            
#             if dist_1 < dist_2:
#                 print("4")
#                 if 'Unknown' in current_label:
#                     label = Add_Person_To_DB(current_emb,False)
                    
#                     prev_labels.append(label)
#                     current_person['Label'] = label
#                     current_person['Status'] = 'Identified'
#                     current_person['Last_PID_Run'] = datetime.now()
                    
#                     return current_person,label,current_emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False
                
#                 else:
#                     distns,emb = Person_ReIdentification(current_person['Img'],flag_known=False,flag_unknown=True)
#                     label,dist,flag = Check_Similarity(distns,thresh)
                    
#                     print(f"1 - current label: {current_label} - label: {label}")
                    
#                     if flag: return Check_And_Go(current_person,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
#                     else:
#                         label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
#                         print(f"1 - current label: {current_label} - label: {label}")

#                         if flag: return Check_And_Go(current_person,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
#                         else: 
#                             print("5")
#                             label = Add_Person_To_DB(emb,False)
                            
#                             prev_labels.append(label)
#                             current_person['Label'] = label
#                             current_person['Status'] = 'Identified'
#                             current_person['Last_PID_Run'] = datetime.now()
                            
#                             return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False
                        
#         else:    
#             print("10")
#             if ('Unknown' in current_label) and not ('Unknown' == current_label):
#                 print("6")
#                 distns,emb = Person_ReIdentification(new_persons[indx]['Img'],flag_known=True,flag_unknown=False)
#                 label,dist,flag = Check_Similarity(distns,thresh)
                
#                 if flag: print("7")
#                 else:
#                     distns,emb = Person_ReIdentification(new_persons[indx]['Img'],flag_known=False,flag_unknown=True)
#                     label,dist,flag = Check_Similarity(distns,thresh)
#                     print(f"2 - current label: {current_label} - label: {label}")
                    
#                     if flag: pass
#                     else: 
#                         label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
#                         print(f"2 - current label: {current_label} - label: {label}")
                        
#                 if dist < current_dist:     
#                     print("8")
#                     label = Add_Person_To_DB(current_emb,False)

#                     prev_labels.append(label)
#                     current_person['Label'] = label
#                     current_person['Status'] = 'Identified'
#                     current_person['Last_PID_Run'] = datetime.now()

#                     return current_person,label,current_emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False  

#                 else:
#                     print("9")
#                     current_person['Label'] = label
#                     current_person['Status'] = 'Identified'
#                     current_person['Last_PID_Run'] = datetime.now()

#                     label = Add_Person_To_DB(emb,False)

#                     prev_labels.append(label)
#                     new_persons[indx]['Label'] = label
#                     new_persons[indx]['Status'] = 'Identified'
#                     new_persons[indx]['Last_PID_Run'] = datetime.now()

#                     return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False  
                
#             else:
#                 print("10")
#                 persons_dists = []
#                 persons_dists.append(current_dist)
                
#                 distns,_ = Person_ReIdentification(new_persons[indx]['Img'],flag_known=True,flag_unknown=False)
#                 try: indx,dist = min(distns, key=lambda x: x[1])
#                 except ValueError: dist = 9e9
                    
#                 persons_dists.append(dist)   
                    
#                 arg = np.argmin(persons_dists)
#                 if arg == 0:
#                     current_person['Label'] = label
#                     current_person['Status'] = 'Identified'
#                     current_person['Last_PID_Run'] = datetime.now()
                    
#                     person_to_investigate = new_persons[indx]
                    
#                 else: person_to_investigate = current_person
                    
#                 distns,emb = Person_ReIdentification(person_to_investigate['Img'],flag_known=False,flag_unknown=True)
#                 label,dist,flag = Check_Similarity(distns,thresh)
#                 print(f"3 - current label: {current_label} - label: {label}")
#                 if flag: return Check_And_Go(person_to_investigate,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
#                 else:
#                     label,dist,flag = Check_Similarity(distns,thresh+thresh_param)
#                     print(f"3 - current label: {current_label} - label: {label}")
#                     if flag: return Check_And_Go(person_to_investigate,label,emb,'Unknown',dist,prev_labels,old_frame,new_persons)
#                     else: 
#                         label = Add_Person_To_DB(emb,False)
                        
#                         prev_labels.append(label)
#                         person_to_investigate['Label'] = label
#                         person_to_investigate['Status'] = 'Identified'
#                         person_to_investigate['Last_PID_Run'] = datetime.now()
                        
#                         return current_person,label,emb,'Unknown',9e9,prev_labels,old_frame,new_persons,False
#     else: 
#         print("2")
#         prev_labels.append(current_label)
#         return current_person,current_label,current_emb,state,current_dist,prev_labels,old_frame,new_persons,True